In [7]:
import pyspark
from pyspark.sql import SparkSession
import os

# fs.s3a.path.style.access true
# dremio.s3.compat true
# fs.s3a.endpoint http://minio:9000

def get_spark_session():

  jar_packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.95.0",
    "software.amazon.awssdk:bundle:2.17.178",
    "software.amazon.awssdk:url-connection-client:2.17.178"
  ]

  spark_extensions = [
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
  ]

  conf = (
    pyspark.SparkConf()
      .setAppName('iceberg_hello_world')
			.set('spark.jars.packages', ','.join(jar_packages))
			.set('spark.sql.extensions', ','.join(spark_extensions))
      .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
      .set('spark.sql.catalog.nessie.uri', os.getenv("NESSIE_URI"))
      .set('spark.sql.catalog.nessie.ref', 'main')
      .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
      .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
      .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
      .set('spark.sql.catalog.nessie.s3.endpoint', os.getenv("MINIO_HOST"))
      .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
      .set('spark.hadoop.fs.s3a.access.key', os.getenv("MINIO_ACCESS_KEY"))
      .set('spark.hadoop.fs.s3a.secret.key', os.getenv("MINIO_SECRET_KEY"))
      .set('spark.hadoop.fs.s3a.path.style.access', 'true')
      .set('spark.hadoop.fs.s3a.connection.timeout', '600000')
      .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
      .set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
      .set('spark.hadoop.fs.s3a.endpoint', os.getenv("MINIO_HOST"))

  )
  return SparkSession.builder.config(conf=conf).master("spark://spark-master:7077").getOrCreate()

spark.sql("CREATE TABLE IF NOT EXISTS nessie.test (id INT, name STRING) USING iceberg")


spark = get_spark_session()
print("OII")
spark.sql("SHOW TABLES IN nessie").show()
spark.sql("SELECT * FROM nessie.test").show()

spark.sql("INSERT INTO nessie.test VALUES (30, 'maria')").show()




OII
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |     test|      false|
+---------+---------+-----------+

+---+----+
| id|name|
+---+----+
+---+----+



24/09/28 01:53:07 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 8) (172.18.0.2 executor 0): java.lang.NoSuchMethodError: 'void software.amazon.awssdk.utils.IoUtils.closeQuietly(java.lang.AutoCloseable, software.amazon.awssdk.thirdparty.org.slf4j.Logger)'
	at software.amazon.awssdk.core.util.SdkUserAgent.kotlinVersion(SdkUserAgent.java:173)
	at software.amazon.awssdk.core.util.SdkUserAgent.getAdditionalJvmLanguages(SdkUserAgent.java:123)
	at software.amazon.awssdk.core.util.SdkUserAgent.getUserAgent(SdkUserAgent.java:98)
	at software.amazon.awssdk.core.util.SdkUserAgent.initializeUserAgent(SdkUserAgent.java:81)
	at software.amazon.awssdk.core.util.SdkUserAgent.<init>(SdkUserAgent.java:51)
	at software.amazon.awssdk.core.util.SdkUserAgent.create(SdkUserAgent.java:58)
	at software.amazon.awssdk.core.client.builder.SdkDefaultClientBuilder.lambda$mergeGlobalDefaults$2(SdkDefaultClientBuilder.java:259)
	at software.amazon.awssdk.utils.builder.SdkBuilder.applyMutation(SdkBuilder.java:6

Py4JJavaError: An error occurred while calling o50.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 11) (172.18.0.2 executor 0): java.lang.NoSuchMethodError: 'void software.amazon.awssdk.utils.IoUtils.closeQuietly(java.lang.AutoCloseable, software.amazon.awssdk.thirdparty.org.slf4j.Logger)'
	at software.amazon.awssdk.core.util.SdkUserAgent.kotlinVersion(SdkUserAgent.java:173)
	at software.amazon.awssdk.core.util.SdkUserAgent.getAdditionalJvmLanguages(SdkUserAgent.java:123)
	at software.amazon.awssdk.core.util.SdkUserAgent.getUserAgent(SdkUserAgent.java:98)
	at software.amazon.awssdk.core.util.SdkUserAgent.initializeUserAgent(SdkUserAgent.java:81)
	at software.amazon.awssdk.core.util.SdkUserAgent.<init>(SdkUserAgent.java:51)
	at software.amazon.awssdk.core.util.SdkUserAgent.create(SdkUserAgent.java:58)
	at software.amazon.awssdk.core.client.builder.SdkDefaultClientBuilder.la

In [13]:
# ## LOAD A CSV INTO AN SQL VIEW
# csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
# csv_df.createOrReplaceTempView("csv_open_2023")

# ## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
# spark.sql("CREATE TABLE IF NOT EXISTS nessie.open_2023 USING iceberg AS SELECT * FROM csv_open_2023").show()
# spark.sql("SELECT COUNT(*) FROM nessie.open_2023").show()

++
||
++
++

+--------+
|count(1)|
+--------+
|  302231|
+--------+

